# 제1고지: 미분 자동 계산

## Step 1. 상자로서의 변수

### 1.1 변수란

### 1.2 Variable Class 구현

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data

In [2]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

x.data = np.array(2.0)
print(x.data)

1.0
2.0


### 1.3 넘파이의 다차원 배열

In [3]:
import numpy as np

x = np.array(1)
x.ndim

0

In [4]:
x = np.array([1, 2, 3])
x.ndim

1

In [5]:
x = np.array([[1, 2, 3],
              [4, 5, 6]])
x.ndim

2

## Step 2. 변수를 낳는 함수

### 2.1 함수란

### 2.2 Function 클래스 구현

In [6]:
class Function:
    def __call__(self, input):
        x = input.data # 데이터를 꺼냄
        y = x ** 2 # 실제계산
        output = Variable(y) # Variable 형태로 되돌린다.
        return output

### 2.3 Function 클래스 이용

In [7]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y)) # type() 함수는 객체의 클래스를 알려준다
print(y.data)

<class '__main__.Variable'>
100


In [8]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError()

In [9]:
class Square(Function):
    def forward(self, x):
        return x ** 2

In [10]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## Step. 3 함수연결

### 3.1 Exp 함수 구현
우선 DeZero에 새로운 함수를 하나 구현하겠습니다. \
바로 $y=e^x$ 이라는 계산을 하는 함수입니다. \
여기서 e는 자연로그의 밑$^{\textrm{base of the natural logarithm}}$ 으로 구체적인 값은 2.718...입니다. \
(오일 러의 수 Eulers number 혹은 네이피어 상수Napiers constan 라고도 합니다)

In [11]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

### 3.2 함수 연결

In [12]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## Step. 4 수치 미분

### 4.1 미분이란
미분이란 무엇일까요? 간단히 말하면 '변화율'을 뜻합니다.
$$ f'(x) = \lim_{h\rightarrow 0} \frac{f(x+h)-f(x)}{h} $$
미세한 차이를 이용 하여 함수의 변화량을 구하는 방법을 수치 미분$^{\textrm{numerical differentiaton}}$이라고 합니다.\
수치 미분은 작은 값을 사용하여 '진정한 미분'을 근사합니다. \
따라서 값에 어쩔 수 없이 오차가 포함되는데, \
이 근사 오차를 줄이는 방법으로는 **"중앙차분$^{\textrm{centered difference}}$"** 이라는 게 있습니다.

### 4.2 수치 미분 구현

In [13]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


### 4.3 합성 함수의 미분

In [14]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


### 4.4 수치 미분의 문제점

## Step. 5 역전파 이론

### 5.1 연쇄 법칙
역전파를 이해하는 열쇠는 **연쇄 법칙** $^{\textrm{chain rule}}$ 입니다.

### 5.2 역전파 원리 도출

### 5.3 계산 그래프로 살펴보기

## Step 6. 수동 역전파

### 6.1 Variable 클래스 추가 구현

In [15]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

### 6.2 Function 클래스 추가 구현


In [16]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input  # 입력 변수를 기억(보관)한다.
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()

### 6.3 Square와 Exp 클래스 추가 구현

In [17]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [18]:
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

### 6.4 역전파 구현

In [19]:
A = Square() 
B = Exp()
C = Square()

x = Variable(np.array(0.5)) 
a = A(x)
b = B(a)
y = C(b)

In [20]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


## Step. 7 역전파 자동화

### 7.1 역전파 자동화의 시작

In [21]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func